In [1]:
%load_ext autoreload
%autoreload 2   

In [2]:
from lib.parse_utils import *
from lib.ocr_utils import *
from lib.pipeline_runner import *

company_id = Please check it in the DB

In [3]:
company_id = 1
country = "USA"
processed_date = pd.Timestamp.now().strftime("%Y-%m-%d")

<a id="rg"></a>

---

<h3>Reading file path</h3>

---

In [30]:
# Define project-relative base directory (portable)
base_dir = os.path.join(os.getcwd(), "data")

# Define the search pattern for all company folders
path_pattern = os.path.join(base_dir, "*23*/*/*ds *s*")

# Collect all PDF paths recursively, ignoring ".ini" files
path = [
    p.replace("\\", "/")
    for p in glob(path_pattern, recursive=True)
    if ".ini" not in p
]

# Show the first few paths
path[:]

['/home/mauricio/git/pdf-etl-pipeline_new/data/2023-nc-d1/2023-08-16_nc_d1/8_16_2023 Bids As Read Summary.pdf',
 '/home/mauricio/git/pdf-etl-pipeline_new/data/2023-nc-d1/2023-03-15_nc_d1/DA00543_Bids As Read.pdf',
 '/home/mauricio/git/pdf-etl-pipeline_new/data/2023-nc-d1/2023-12-06_nc_d1/Bids As Read.pdf']

In [31]:
df = run_extraction_pipeline(path, company_id=company_id, country=country, processed_date=processed_date)


📂 Processing file: /home/mauricio/git/pdf-etl-pipeline_new/data/2023-nc-d1/2023-08-16_nc_d1/8_16_2023 Bids As Read Summary.pdf
📄 Extracted 0 text lines from PDF.
8_16_2023 bids as read summary.pdf
🔎 Detected doc_type=nc_bids_as_read
   ➜ NC Bids As Read parsed 1 rows
[{'company_id': 1, 'country': 'USA', 'processed_date': '2025-11-17', 'file': PosixPath('/home/mauricio/git/pdf-etl-pipeline_new/data/2023-nc-d1/2023-08-16_nc_d1/8_16_2023 Bids As Read Summary.pdf'), 'doc_type': 'nc_bids_as_read', 'state_header': None, 'owner_header': None, 'letting_date': None, 'letting_time': None, 'contract_id': None, 'description': None, 'counties': None, 'engineers_estimate': None, 'total_bids_received': None, 'contractor_name': None, 'amount_bid': None}]
   ➜ parse_document_by_type returned <class 'list'> len=1

📂 Processing file: /home/mauricio/git/pdf-etl-pipeline_new/data/2023-nc-d1/2023-03-15_nc_d1/DA00543_Bids As Read.pdf
📄 Extracted 0 text lines from PDF.
da00543_bids as read.pdf
🔎 Detected doc

,company_id,country,processed_date,file,doc_type,state_header,owner_header,letting_date,letting_time,contract_id,description,counties,engineers_estimate,total_bids_received,contractor_name,amount_bid
0,1,USA,2025-11-17,/home/mauricio/git/pdf-etl-pipeline_new/data/2023-nc-d1/2023-08-16_nc_d1/8_16_2023 Bids As Read Summary.pdf,nc_bids_as_read,None,None,None,None,None,None,None,None,None,None,None
1,1,USA,2025-11-17,/home/mauricio/git/pdf-etl-pipeline_new/data/2023-nc-d1/2023-03-15_nc_d1/DA00543_Bids As Read.pdf,nc_bids_as_read,None,None,None,None,None,None,None,None,None,None,None
2,1,USA,2025-11-17,/home/mauricio/git/pdf-etl-pipeline_new/data/2023-nc-d1/2023-12-06_nc_d1/Bids As Read.pdf,nc_bids_as_read,None,None,None,None,None,None,None,None,None,None,None


In [32]:
df.head(10)

,company_id,country,processed_date,file,doc_type,state_header,owner_header,letting_date,letting_time,contract_id,description,counties,engineers_estimate,total_bids_received,contractor_name,amount_bid
0,1,USA,2025-11-17,/home/mauricio/git/pdf-etl-pipeline_new/data/2023-nc-d1/2023-08-16_nc_d1/8_16_2023 Bids As Read Summary.pdf,nc_bids_as_read,None,None,None,None,None,None,None,None,None,None,None
1,1,USA,2025-11-17,/home/mauricio/git/pdf-etl-pipeline_new/data/2023-nc-d1/2023-03-15_nc_d1/DA00543_Bids As Read.pdf,nc_bids_as_read,None,None,None,None,None,None,None,None,None,None,None
2,1,USA,2025-11-17,/home/mauricio/git/pdf-etl-pipeline_new/data/2023-nc-d1/2023-12-06_nc_d1/Bids As Read.pdf,nc_bids_as_read,None,None,None,None,None,None,None,None,None,None,None


In [27]:

mask_none = df.isna().any(axis=1)
df_issues = df[mask_none].copy()

pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", None)
df_issues.head(10)


,company_id,country,processed_date,file,doc_type,contract_id,state_header,owner_header,project_number,county,description,vendor,letting_date,award_value


---
Connecting to the database (Postgres)

In [ ]:
from db_connection import postgres
db = postgres()

---
Uploading the files to the database (Postgres)

In [ ]:
if df.empty:
    print("✅ No new invoices to upload — all records already exist.")
else:
    print(f"🚀 Uploading {len(df)} new invoices to the database...")

    inserted_count = db.insert_dataframe(df, "invoices")

    if inserted_count > 0:
        print(f"✅ Successfully uploaded {inserted_count} new invoices to PostgreSQL.")
    elif inserted_count == 0:
        print("❌ No new invoices were uploaded — all were duplicates.")
    else:
        print("❌ Upload failed — check error log above.")


---